<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [9]:
import spacy

nlp = spacy.load("fr_core_news_sm")  # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [10]:
# Veuillez écrire votre code ici.
print(f"Liste des traitements disponibles : {nlp.pipe_names}")

for pipe in nlp.pipe_names:
    if pipe not in ["tok2vec", "morphologizer", "parser"]:
        nlp.disable_pipe(pipe)

print(f"Liste des traitements actifs : {nlp.pipe_names}")

Liste des traitements disponibles : ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Liste des traitements actifs : ['tok2vec', 'morphologizer', 'parser']


In [11]:
from spacy.lang.fr.examples import sentences

s1 = sentences[2]  # prenons la 3e phrase comme exemple
s1

"San Francisco envisage d'interdire les robots coursiers sur les trottoirs"

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [25]:
# Veuillez écrire votre code et votre réponse ici.
s1_doc = nlp(s1)

print("Tokens:")
for token in s1_doc:
    print(f"{token.text} [{token.pos_}] [{token.dep_}]")

Tokens:
San [PRON] [nsubj]
Francisco [PROPN] [flat:name]
envisage [VERB] [ROOT]
d' [ADP] [case]
interdire [NOUN] [obl:arg]
les [DET] [det]
robots [NOUN] [obj]
coursiers [ADJ] [amod]
sur [ADP] [case]
les [DET] [det]
trottoirs [NOUN] [nmod]


**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`.

In [24]:
# Veuillez écrire votre code ici.
print("Groupes de mots:")
for token in s1_doc:
    if token.dep_ in ["nsubj", "obj"]:
        print("----------------")
        print(f"{token.text} [{token.pos_}] [{token.dep_}]:")
        for subtoken in token.subtree:
            print(f"- {subtoken.text} [{token.pos_}] [{subtoken.dep_}]")

Groupes de mots:
----------------
San [PRON] [nsubj]:
- San [PRON] [nsubj]
- Francisco [PRON] [flat:name]
----------------
robots [NOUN] [obj]:
- les [NOUN] [det]
- robots [NOUN] [obj]
- coursiers [NOUN] [amod]
- sur [NOUN] [case]
- les [NOUN] [det]
- trottoirs [NOUN] [nmod]


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

> Selon la documentation officielle du format: https://universaldependencies.org/format.html
>
> - Les relations de dépendances sont situées dans la 8e colonne (DEPREL) et sont représentées avec le format des étiquettes de dépendances universelles (https://universaldependencies.org/u/dep/index.html).
> - La 7e colonne (HEAD) contient l'identifiant du token gouverneur (ou 0 si c'est la racine).
> - La 9e colonne (DEPS) contient, elle, optionnellement, un graphe de dépendances amélioré sous forme de paires (HEAD, DEPREL)


In [26]:
from spacy.tokens import DocBin, Doc

test_data = DocBin().from_disk("../pw2/spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [74]:
# Veuillez écrire votre code ici.
doc = list(test_data.get_docs(nlp.vocab))[1]
s2 = list(doc.sents)[6]
s2

Trois ans plus tard, il tient un discours sur la crise.

**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [65]:
from spacy import displacy
from IPython.display import HTML, display

In [118]:
# Veuillez écrire votre code ici.
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(s2, style="dep", options={"compact": True}, jupyter=False)
)))

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [114]:
# Veuillez écrire votre code ici.
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(nlp(s2.as_doc()), style="dep", options={"compact": True}, jupyter=False)
)))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

> La phrase contient 13 tokens, dont 2 tokens de ponctuation et 10 relations entre les tokens.
>
> La pipeline `nlp` génère également 10 relations. Il y a toutefois deux erreurs:
>   - le mot "ans" devrait dépendre de "tient" mais il dépend que de "plus" avec le résultat de la pipeline `nlp`.
>   - le mot "tard" devrait dépendre de "ans" mais il dépend de "tien" avec le résultat de la pipeline `nlp`.
>
> Le taux de correction est donc de 11/13 = 0.846.

**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [69]:
from spacy.scorer import Scorer
from spacy.training import Example

In [120]:
# Veuillez écrire votre code et votre réponse ici.
score = Scorer().score_deps([Example(nlp(s2.as_doc()), s2.as_doc())], "dep")
print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.846
LAS: 0.846


> Le score est légèrement différent de celui qui a été calculé à la main, certainement parce que le scorer établit un f-score sur une comparaison plus complète

## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [144]:
# Veuillez écrire votre code ici.
score = Scorer().score([
    Example(
        nlp(Doc(nlp.vocab, [token.text for token in doc])),
        doc
    ) for doc in test_data.get_docs(nlp.vocab)
])
print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.801
LAS: 0.675


> La pipeline indique un score de 0.88 pour le UAS et 0.84 pour le LAS, nos scores pour le unlabelled sont assez similaires, mais pour le labelled, il y a une plus grande différence. On peut attribuer ça au fait que le modèle utilisé n'a pas été entrainé sur le même corpus que celui que nous utilisons dans nos tests. Ainsi, il se peut que les relations de dépendances ne soient pas attribuées de la même manière.
>
> Le labelling étant plus précis, il est normal que le score soit plus bas si le modèle n'a pas assez d'informations contextuelles pour attribuer les relations de dépendances correctement.

**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [150]:
# Veuillez écrire votre code ici.
from tabulate import tabulate

dep_las_per_type = score["dep_las_per_type"]
dep_las_per_type = sorted(dep_las_per_type.items(), key=lambda x: x[1]["f"], reverse=True)

print(tabulate(
    [(dep, f"{score['p']:.3f}", f"{score['r']:.3f}", f"{score['f']:.3f}") for dep, score in dep_las_per_type],
    headers=["Dep", "Precision", "Recall", "F1"],
    tablefmt="fancy_grid"
))

╒══════════════╤═════════════╤══════════╤═══════╕
│ Dep          │   Precision │   Recall │    F1 │
╞══════════════╪═════════════╪══════════╪═══════╡
│ det          │       0.935 │    0.937 │ 0.936 │
├──────────────┼─────────────┼──────────┼───────┤
│ case         │       0.876 │    0.923 │ 0.899 │
├──────────────┼─────────────┼──────────┼───────┤
│ cc           │       0.805 │    0.798 │ 0.802 │
├──────────────┼─────────────┼──────────┼───────┤
│ aux:pass     │       0.636 │    0.925 │ 0.754 │
├──────────────┼─────────────┼──────────┼───────┤
│ cop          │       0.734 │    0.755 │ 0.745 │
├──────────────┼─────────────┼──────────┼───────┤
│ root         │       0.695 │    0.8   │ 0.744 │
├──────────────┼─────────────┼──────────┼───────┤
│ nummod       │       0.763 │    0.726 │ 0.744 │
├──────────────┼─────────────┼──────────┼───────┤
│ mark         │       0.714 │    0.758 │ 0.735 │
├──────────────┼─────────────┼──────────┼───────┤
│ nsubj:pass   │       0.661 │    0.82  │ 0.732 │


## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [152]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [153]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.

# !python -m spacy train config.cfg \
#     --output ./myDEPparser1-5epoch \
#     --paths.train ../pw2/spacy_data/fr-ud-train.spacy \
#     --paths.dev ../pw2/spacy_data/fr-ud-dev.spacy \
#     --verbose

[2025-03-27 15:22:50,247] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
✔ Created output directory: myDEPparser1-5epoch
ℹ Saving to output directory: myDEPparser1-5epoch
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-27 15:22:54,754] [INFO] Set up nlp object from config
[2025-03-27 15:22:54,768] [DEBUG] Loading corpus from path: ../pw2/spacy_data/fr-ud-dev.spacy
[2025-03-27 15:22:54,769] [DEBUG] Loading corpus from path: ../pw2/spacy_data/fr-ud-train.spacy
[2025-03-27 15:22:54,770] [INFO] Pipeline: ['tok2vec', 'morphologizer', 'parser']
[2025-03-27 15:22:54,773] [INFO] Created vocabulary
[2025-03-27 15:22:54,773] [INFO] Finished initializing nlp object
[2025-03-27 15:23:37,533] [INFO] Initialized pipeline components: ['tok2vec', 'morphologizer', 'parser']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-27 15:23:37,547] [DEBUG] Loading corpus from path: .

> Nous avons laissé la configuration par défaut qui utilise un paramètre de "patience" pour déterminer lorsque l'entraînement doit s'arrêter. Cela a pris 5 époques pour que le modèle atteigne son meilleur score sur le jeu de validation.


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [155]:
# Veuillez écrire votre code ici.
nlp2 = spacy.load("./myDEPparser1-5epoch/model-best")
test_data = DocBin().from_disk("../pw2/spacy_data/fr-ud-test.spacy")

score = Scorer().score([
    Example(
        nlp2(Doc(nlp2.vocab, [token.text for token in doc])),
        doc
    ) for doc in test_data.get_docs(nlp2.vocab)
])

print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.877
LAS: 0.832


> Le résultat du UAS est similaire, on voit par contre une grande augmentation dans le résultat du LAS. Ce qui confirme l'hypothèse que le modèle prêt-à-l'emploi n'a pas été entraîné sur le même corpus que celui que nous utilisons pour nos tests. Et que d'avoir un modèle entrainé sur un corpus similaire permet d'obtenir de meilleurs résultats en termes de labelling des dépendances.

**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [156]:
# Veuillez écrire votre code ici.

dep_las_per_type = score["dep_las_per_type"]
dep_las_per_type = sorted(dep_las_per_type.items(), key=lambda x: x[1]["f"], reverse=True)

print(tabulate(
    [(dep, f"{score['p']:.3f}", f"{score['r']:.3f}", f"{score['f']:.3f}") for dep, score in dep_las_per_type],
    headers=["Dep", "Precision", "Recall", "F1"],
    tablefmt="fancy_grid"
))

╒════════════╤═════════════╤══════════╤═══════╕
│ Dep        │   Precision │   Recall │    F1 │
╞════════════╪═════════════╪══════════╪═══════╡
│ det        │       0.97  │    0.982 │ 0.976 │
├────────────┼─────────────┼──────────┼───────┤
│ case       │       0.921 │    0.962 │ 0.941 │
├────────────┼─────────────┼──────────┼───────┤
│ aux        │       0.94  │    0.874 │ 0.906 │
├────────────┼─────────────┼──────────┼───────┤
│ cc         │       0.898 │    0.888 │ 0.893 │
├────────────┼─────────────┼──────────┼───────┤
│ aux:caus   │       0.917 │    0.846 │ 0.88  │
├────────────┼─────────────┼──────────┼───────┤
│ root       │       0.886 │    0.863 │ 0.875 │
├────────────┼─────────────┼──────────┼───────┤
│ amod       │       0.866 │    0.884 │ 0.874 │
├────────────┼─────────────┼──────────┼───────┤
│ obj        │       0.828 │    0.901 │ 0.863 │
├────────────┼─────────────┼──────────┼───────┤
│ nsubj      │       0.852 │    0.855 │ 0.853 │
├────────────┼─────────────┼──────────┼─

**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

> On constate que les trois premiers avant étaient `[det, case, cc]` et maintenant ce sont `[det, case, aux]` avec des f-scores plus élevés.
>
> Les trois derniers étaient `[flat:foreign, discourse, csubj]` et maintenant ce sont `[obl:agent, discourse, csubj]` avec des f-scores qui sont toujours à 0.
>
> On notera toutefois qu'il y a moins de labels qui ont un f-score de 0, ce qui est une amélioration par rapport au modèle prêt-à-l'emploi et indique que notre modèle est peut-être plus précis sur les relations de dépendances dans certains contextes spécifiques à notre corpus.
>
> csubj (clausal subject) est un label pour lequel les scores n'augmentent pas avec le parser entraîné, certainement parce que ce label n'est pas utilisé dans notre corpus (dans UD_english, ce label est utilisé pour 340 nodes, c'est 0%: https://universaldependencies.org/docs/en/dep/csubj.html)

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.